In [1]:
import os
import shutil

def rename_and_copy_folders(root_dir, dest_dir):
    # Define mapping for renaming
    mapping = {
        "10": "A", "11": "B", "12": "C", "13": "D", "14": "E", "15": "F", 
        "16": "G", "17": "H", "18": "I", "19": "J", "20": "K", "21": "L", 
        "22": "M", "23": "N", "24": "O", "25": "P", "26": "Q", "27": "R", 
        "28": "S", "29": "T", "30": "U", "31": "V", "32": "W", "33": "X", 
        "34": "Y", "35": "Z", "36": "a", "37": "b", "38": "c", "39": "d", 
        "40": "e", "41": "f", "42": "g", "43": "h", "44": "i", "45": "j", 
        "46": "k", "47": "l", "48": "m", "49": "n", "50": "o", "51": "p", 
        "52": "q", "53": "r", "54": "s", "55": "t", "56": "u", "57": "v", 
        "58": "w", "59": "x", "60": "y", "61": "z",
        'x_sampled': '*', 
    '÷_sampled': '÷', 
    '(_sampled': '(', 
    ')_sampled': ')', 
    '+_sampled': '+', 
    '-_sampled': '-'
    }

    for subdir, dirs, files in os.walk(root_dir):
        # Get the name of the current directory
        dir_name = os.path.basename(subdir)
        # Get the new name from mapping
        new_name = mapping.get(dir_name, dir_name)
        # Construct the new directory path
        new_dir_path = os.path.join(dest_dir, new_name)
        # Create the new directory
        os.makedirs(new_dir_path, exist_ok=True)
        # Copy files to the new directory
        for file in files:
            src_file_path = os.path.join(subdir, file)
            dest_file_path = os.path.join(new_dir_path, file)
            shutil.copyfile(src_file_path, dest_file_path)

# Kaggle specific paths
root_directory = "/kaggle/input/balanced-emnist-maths-symbol-dataset/"
destination_directory = "/kaggle/working/renamed_dataset/"
rename_and_copy_folders(root_directory, destination_directory)


In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the root directory of your dataset
dataset_dir = "/kaggle/working/renamed_dataset"

# Define the image size
img_height, img_width = 224, 224

# Prepare data
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

# Define the MobileNetV3 model
base_model = MobileNetV3Small(input_shape=(img_height, img_width, 3),
                              weights='imagenet',
                              include_top=False)

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(68, activation='softmax')(x)  # 68 classes: 0-9, A-Z, a-z, (,), -, +, *, /

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Save the trained model
model.save("/kaggle/working/mobilenetv3_symbol_classifier.h5")

# Function to predict class for a single image
def predict_class(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    class_labels = sorted(train_generator.class_indices.keys())
    predicted_class = class_labels[predicted_class_index]
    return predicted_class

# Example usage: Predicting class for a single image
image_path = "/kaggle/working/renamed_dataset/0/643251.png"  # Adjust with your image path
predicted_class = predict_class(image_path, model)
print("Predicted class:", predicted_class)


Found 60860 images belonging to 68 classes.
Found 15164 images belonging to 68 classes.
Epoch 1/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 181s 77ms/step - accuracy: 0.6309 - loss: 1.1406 - val_accuracy: 0.0148 - val_loss: 8.6852
Epoch 2/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 87s 46ms/step - accuracy: 0.7705 - loss: 0.5520 - val_accuracy: 0.0292 - val_loss: 6.7714
Epoch 3/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 89s 46ms/step - accuracy: 0.7900 - loss: 0.4946 - val_accuracy: 0.3599 - val_loss: 2.5779
Epoch 4/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 90s 47ms/step - accuracy: 0.8015 - loss: 0.4579 - val_accuracy: 0.1600 - val_loss: 3.7245
Epoch 5/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 88s 46ms/step - accuracy: 0.8088 - loss: 0.4402 - val_accuracy: 0.5681 - val_loss: 1.4864
Epoch 6/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 88s 46ms/step - accuracy: 0.8101 - loss: 0.4264 - val_accuracy: 0.5983 - val_loss: 1.2299
Epoch 7/10
1902/1902 ━━━━━━━━━━━━━━━━━━━━ 86s 45ms/step - accuracy: 0.8199 - loss: 0.4037 - val_accuracy: 0.3603 - val_

In [11]:
def predict_class(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    class_labels = sorted(train_generator.class_indices.keys())
    predicted_class = class_labels[predicted_class_index]
    return predicted_class

# Example usage: Predicting class for a single image
image_path = "/kaggle/working/renamed_dataset/v/239328.png"  # Adjust with your image path
predicted_class = predict_class(image_path, model)
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted class: V
